# How is France progressing towards the Dec 15 targets?

(Created on Dec 4)

France has already begun to ease the resrictions for their second confinement that started on Oct 28 2020. 

Further loosening of restrictions is supposedly tied to meeting the following targets by Dec 15 2020:

* daily positive cases under 5000

* ICU occupancy in the 2500-3000 range

These targets were introduced by President Macron in a televised speech 

## Answer: we don't know

The average French person couldn't answer, because there is no simple way to find this out.

I am curious why, with such clear, simple targets, there is no central dashboard to track our progress and _responsabilise_ the French public?

**This is especially important right now, as the target for daily positive cases is looking increasingly unfeasible.** Yet there may still be time to correct course, if enough people are made aware of the problem.

_Even if we miss the targets, **with less than 3 weeks before Christmas and all the travel & gatherings entailed, shouldn't we all be working towards getting new cases as low as possible?**_

## France has the data to track this

The data is not perfect and the reporting of it is inconsistent, but it does exist.

Below is my attempt show how we're doing, as well as to overcome the limitations of the pubicly-available data.

[Jump explanation of daily cases target (at risk)](#Target-1:-Daily-positive-cases-%3C-5000)

[Jump to ICU target (still OK for now)](http://localhost:8888/notebooks/covid19/A_targets_dec15.ipynb#Target-2:--ICU-occupancy-2500---3000)


# Setup

## Load libraries

In [94]:
import numpy as np
import pandas as pd
import dataframe_image as dfi

#import pprint as pp
from datetime import datetime, timedelta

%load_ext autoreload
%autoreload 2

#to display offline interactive plots
import plotly as py

# mapping library
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
#from urllib.request import urlopen
#import json

# to export plots to html
#import plotly.io as pio

import plotly.io as pio
pio.renderers.default='notebook'

# hide annoying repeated deprec warnings (statsmodel issue)
import warnings
warnings.simplefilter('once', category=UserWarning)

# load my data processing modules
import process_test_data as pt
import process_hosp_data as hd
import process_kpi as kpi
from process_metro_bubble import to_chart_studio

# fix slowness issue? See https://github.com/microsoft/vscode-python/issues/10998
%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



## List data sources

In [95]:
## Data sources
# TODO: make into dict with metrics as keys

source_incid = kpi.source
source_rea = hd.source
source_proc = 'https://github.com/limegimlet/covid19'

print("Sources for raw data & processing:\n")
for source in [source_incid, source_rea, source_proc]:
    print(source)

Sources for raw data & processing:

https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/
https://www.data.gouv.fr/en/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/
https://github.com/limegimlet/covid19


## Create dataframes

In [96]:
# create national pos df
df = pt.create_testing_df().groupby('jour')[['pos']].sum().rolling(7).mean().round()#.reset_index()
#df.iplot()

# get rea raw occ numbers
hosp_df = hd.get_hosp_data()

# create national rea df
rea_fr = hosp_df.groupby('jour').sum()[['rea']]#.reset_index()
#rea_fr.iplot()

# join the two
plot_df = df.join(rea_fr, how='outer').loc['2020-10-28':]
plot_df.columns = ["Positive cases, 7-day avg", 'Covid patients in ICU']
#plot_df = plot_df.stack().reset_index().rename(columns={"level_1":'metric', 0:'value'})
plot_df = plot_df.reset_index()
plot_df.head()

,jour,"Positive cases, 7-day avg",Covid patients in ICU
0,2020-10-28,46152.0,3009
1,2020-10-29,46870.0,3119
2,2020-10-30,47334.0,3334
3,2020-10-31,47488.0,3407
4,2020-11-01,47438.0,3529


## Define helper functions

In [97]:
def df_to_md_table(df):
    '''converts df to markdown. 
    Used for pasting into an .MD file'''
    
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    display(Markdown(df_formatted.to_clipboard(sep="|", index=True)))


def create_projection_line(metric, n, start_x, start_y, end_x, end_y, dash):
    
    trace_proj = go.Scatter(x=[start_x, end_x], 
                            y=[start_y, end_y],  
                            mode='lines', 
                            line_color='darkgray',
                            name='{}-day chg rate'.format(n),
                            #hovertext=end_x,
                            hovertemplate="Est. date: " + end_x,
                            line_dash=dash,
                            line_width=1)
    
    return trace_proj

def project_target(metric, target_val, df, target_date='2020-12-15', date_fmt='%Y-%m-%d'):
    latest_date = df.index.max()
    latest_val = df.loc[latest_date][metric]

    # Required change rate to meet goal
    chg_reqd = int(target_val - latest_val)
    pct_chg_reqd = chg_reqd/latest_val
    days_left = (datetime.strptime(target_date, date_fmt) - datetime.strptime(latest_date, date_fmt)).days
    daily_pct_chg_reqd = round(pct_chg_reqd/days_left, 3)


    # Compare with actual change rate

    print("=== Projected days to reach {} {} ===\n".format(target_val, metric))
    print("To meet this, we need to change daily {} by {} between now ({}) and {}.".format(metric, chg_reqd, latest_date, target_date))
    print("This will require a minimum {} average daily change over next {} days.\n".format(daily_pct_chg_reqd, days_left))

    rows = []
    traces = []
    ndays = [14, 7, 5, 3]
    dashes = ['dot', 'dashdot', 'dash', 'solid']
    n_tups = list(zip(ndays, dashes))
    
    
    # Create traces for plot projections
    for tup in n_tups:
        n = tup[0]
        dash = tup[1]
        n_days_prev = df.index[-n]
        n_days_prev_val = df.loc[n_days_prev][metric]
        chg_actual = latest_val - n_days_prev_val
        pct_chg_actual = chg_actual/n_days_prev_val
        daily_chg_actual = round(pct_chg_actual/n, 3)

        # how long to reach target?
        est_days_target = round(pct_chg_reqd/daily_chg_actual, 0)      
        if np.isinf([est_days_target]) or np.isneginf([est_days_target]):
            print("Warning: Estimated additional days using {}-day change rate is infinite.".format(n) + 
                  "Unable to calculate a target date.")
            est_target_date = '9999-99-99'
        else:
            est_days_target = int(est_days_target)
            est_target_date = datetime.strftime((datetime.strptime(latest_date, date_fmt) + timedelta(est_days_target)), date_fmt)

        #print("Using {}-day chg rate of {}: {} days, or {}\n".format(n, daily_chg_actual, est_days_target, est_target_date))
        n_row = ['{} days'.format(n), daily_chg_actual, est_days_target, est_target_date]
        rows.append(n_row)
        
        # create traces for each row
        trace = create_projection_line(metric, n, n_days_prev, n_days_prev_val, est_target_date, target_val, dash)
        traces.append(trace)

    labels = ['Estimate based on previous', 'Avg. daily change rate', 'Est. days to reach target', 'Est. target date']
    project_df = pd.DataFrame(data=rows, columns=labels)

    return project_df, traces

def plot_progress(plot_col, plot_df, target_trace, proj_traces):
    color_seq = [color_map[plot_col]]
    title = "{}: will it reach its target by Dec 15?".format(plot_col)
    x_min = plot_df['jour'].min()
    x_max = plot_df['jour'].max()
    fig = px.bar(plot_df, x = 'jour', y = plot_col, color_discrete_sequence=color_seq,
                 #facet_row = 'metric', facet_row_spacing=0.15, 
                 range_x=['2020-10-28', '2020-12-15'], title=title)
    
    fig.add_trace(target_trace) # target to reach by dec 15 
    fig.add_traces(proj_traces) # projections using different daily change rates
    fig.update_xaxes(tickangle=45)
    
    return fig

### TODO: Make into functions

## project scenarios for different target values

#metric = 'pos'
#from IPython.display import display

#for target_val in [4500, 4000, 3500, 3000, 2500, 2000]:

    #estimated_pos_dates, pos_traces = project_target(metric, target_val, df)
    #display(estimated_pos_dates)
    #print('\n')
#pos_traces

## export df as image

#fname = "conf_chg_summary.png"
#dfi.export(new_summary,"../covid_dataviz/img/{}".format(fname), )

## Set variables

In [98]:
# define colors
pal = py.colors.qualitative.Plotly
color_map = {"Positive cases, 7-day avg":pal[0], 
             'Covid patients in ICU':pal[1]}

# define constants
pos_target_val = 5000
rea_target_val = 3000
confined = '2020-10-28'
target_date = '2020-12-15'
x_min = plot_df['jour'].min()

# create hlines to show target for each metric on plots
pos_target = go.Scatter(x=[x_min, target_date], 
                            y=[pos_target_val, pos_target_val],  
                            mode='lines', 
                            line_color='black',
                            name='Cases target',
                            line_dash = 'dash')

rea_target = go.Scatter(x=[x_min, target_date], 
                            y=[rea_target_val, rea_target_val],  
                            mode='lines', 
                            line_color='black',
                            name='ICU target',
                            line_dash='dot')

# Target 1: Daily positive cases < 5000

Positive tests numbers are stagnating for several days now, unlike the rapid decrease seen at the beginning of the second confinement. 

Yet we still need to nearly halve the current number of daily positive tests, which of course becomes increasigly unrealistic the longer they stagnate.

In [114]:
# project days to reach target using recent avg daily change rates

metric = 'pos'
target_val = 5000

estimated_pos_dates, pos_traces = project_target(metric, target_val, df)
display(estimated_pos_dates)

=== Projected days to reach 5000 pos ===

To meet this, we need to change daily pos by -3652 between now (2020-12-01) and 2020-12-15.
This will require a minimum -0.03 average daily change over next 14 days.



,Estimate based on previous,Avg. daily change rate,Est. days to reach target,Est. target date
0,14 days,-0.041,10,2020-12-11
1,7 days,-0.027,16,2020-12-17
2,5 days,-0.017,25,2020-12-26
3,3 days,-0.021,20,2020-12-21


In [115]:
# show on plot

plot_col = "Positive cases, 7-day avg"
target_trace = pos_target
proj_traces = pos_traces

pos_fig = plot_progress(plot_col, plot_df, target_trace, proj_traces)
pos_fig.update_layout(title="<b>{}</b>: <br>Decrease has stopped. If trend continues, 5000 cases impossible by Dec 15".format(plot_col))
pos_fig.show()
fname = "{}_progress".format(metric)
to_chart_studio(pos_fig, fname)

***Not shown above: this trend continues to at least Dec 4***

In other words, we still have the same number of daily positives to shed (~4000) within an ever-narrowing time frame.

## Fill in the gap for previous 3 days

The challenge with reporting on the Santé Publique France (SPF) data on data.gouv.fr is that they have a 3-day lag. That's why as of the evening of Dec 3, the positve case numbers shown here only go up to Nov 30. 

However, I can manually get the raw daily numbers from the nighly updates on the [Covid-19 page of the SPF website](https://www.santepubliquefrance.fr/dossiers/coronavirus-covid-19/coronavirus-chiffres-cles-et-evolution-de-la-covid-19-en-france-et-dans-le-monde#block-266151). Every night around 20h they update the site with new positive tests reported in the previous 24 hours. 

I've begun taking screenshots. Yes, screenshots, because this info gets overwritten every night.

(And no, in answer to you codeaholics our there: I don't want to futz with creating a webscraper for 2 bullet points, at least not yet).

![dec 2](images/dec2_spf_update.png)

![dec 3](images/dec3_spf_update.png)

![dec 4](images/dec4_spf_update.png)

Incidentally, this is also the only way to get numbers for positive **antigen** tests, which are not shown in my plots, or anyone else's, since only PCR test results are available to the public as downloadable data. 

Anyways, here are the numbers from these cutting-edge screenshots:

In [101]:
# manually create df of latest SPF updates

dates = ['2020-11-30', '2020-12-01', '2020-12-02', '2020-12-03', '2020-12-04']
day_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
pcr_pos = [2483, 6201, 12071, 10657, 9002]
antigen_pos = [1522, 1882, 1993, 2039, 2219]

spf_updates = pd.DataFrame({'jour': dates, 'day': day_week, 'pos_pcr': pcr_pos, 'pos_antigen': antigen_pos})
spf_updates

,jour,day,pos_pcr,pos_antigen
0,2020-11-30,Monday,2483,1522
1,2020-12-01,Tuesday,6201,1882
2,2020-12-02,Wednesday,12071,1993
3,2020-12-03,Thursday,10657,2039
4,2020-12-04,Friday,9002,2219


Now, let's plot PCR positives from Nov 30 onwards. 

Note the positive PCR tests are the "raw" daily numbers, not rolling 7-day averages. 

## Plot SPF nightly numbers

In [102]:
title = 'Daily raw counts of positive PCR tests + antigen tests since Mon Nov 30'
spf_updates.set_index('day')[['pos_pcr', 'pos_antigen',]].iplot(kind='bar', 
                                    barmode='stack', 
                                   title=title)

### Why the daily positives vary so widely

Probably the first thing you notice is how much these fluctuate. This is the weekend effect. 

It happens every week: the new positives recorded for Saturday and Sunday, which SPF will report 24 hours later on Sunday & Monday, respectively, will be always lower. 

Meanwhile, at the labs there is furious data-entry for test results that piled up over the weekend. As a result, SPF's nightly updates for Tuesday and Wednesday will also include catching up on the weekend backlog.

### The solution: use rolling 7-day averages

This is why typically Covid plots (including mine) focus on the rolling 7-day average for new cases: that is, take the sum the preceding 7 days' of test numbers, then divide by 7 to get a "smoothed" number that makes the overall trend much more clear.

<blockquote>This day-to-day 'noisiness' is important to keep in mind when reading, watching, or listening to media reports on Covid in your community. If a number for new cases seems like a big drop or jump, it's likely it's the raw number, which does NOT account for weekend under-reporting and then the Tuesday and Wednesday catching-up.</blockquote>

In French these are ususally indicated by _sur semaine glissante_.

### Calculate rolling averages of PCR positives for last 3 days

Since we already know the rollling average up to Dec 1, as well as the raw PCR positive counts for Dec 2 to 4, we can now calculate the 7-day rolling averages. 

`current_7d_avg = (previous_7d_avg * 6) + current_raw_ / 7`

I won't bother doing this for antigen tests as it's too early to tell if they suffer from the weekend effect as well. 

In [103]:
rolling = plot_df[['jour', plot_col]].rename(columns={plot_col: 'rolling_pcr_pos'}).tail().reset_index(drop=True)
spf_updates = spf_updates.merge(rolling, on='jour', how='left')
spf_updates

,jour,day,pos_pcr,pos_antigen,rolling_pcr_pos
0,2020-11-30,Monday,2483,1522,8887.0
1,2020-12-01,Tuesday,6201,1882,8652.0
2,2020-12-02,Wednesday,12071,1993,NaN
3,2020-12-03,Thursday,10657,2039,NaN
4,2020-12-04,Friday,9002,2219,NaN


In [104]:
# calculate the missing rolling 7-day avgs

def fill_missing_rolling_avg(spf_updates):
    '''Calculates missing rolling averages using last known rolling average
    and subsequent days' raw counts'''
    
    empty_rows = spf_updates.loc[spf_updates['rolling_pcr_pos'].isnull()].index.values
    for row in empty_rows:
        prev_roll_pcr_pos = spf_updates.loc[row-1]['rolling_pcr_pos']
        spf_updates.iloc[row, 4] = int(((prev_roll_pcr_pos * 6) + spf_updates.loc[row]['pos_pcr'])/7)
    
    return spf_updates

latest_rolling_pos_date = plot_df.dropna()['jour'].max() 
latest_rolling_pos_val = plot_df.loc[plot_df['jour']==latest_rolling_pos_date][plot_col].values[0]

#spf_updates.loc[0, 'rolling_pcr_pos'] = latest_rolling_pos_val
#spf_updates.loc[1:, 'rolling_pcr_pos'] = np.nan

spf_updates = fill_missing_rolling_avg(spf_updates)
spf_updates

,jour,day,pos_pcr,pos_antigen,rolling_pcr_pos
0,2020-11-30,Monday,2483,1522,8887.0
1,2020-12-01,Tuesday,6201,1882,8652.0
2,2020-12-02,Wednesday,12071,1993,9140.0
3,2020-12-03,Thursday,10657,2039,9356.0
4,2020-12-04,Friday,9002,2219,9305.0


To keep things clean, let's now drop the raw PCR positives:

In [105]:
spf_updates = spf_updates.drop('pos_pcr', axis=1)
spf_updates

,jour,day,pos_antigen,rolling_pcr_pos
0,2020-11-30,Monday,1522,8887.0
1,2020-12-01,Tuesday,1882,8652.0
2,2020-12-02,Wednesday,1993,9140.0
3,2020-12-03,Thursday,2039,9356.0
4,2020-12-04,Friday,2219,9305.0


OK time to plot again, this time the rolling PCR positives.

## Plot for same days using 7-d rolling averages

In [106]:
title = "Once you look at the <b>rolling 7-day average</b>, daily PCR postives are stagnating"
rolling_pcr = spf_updates.set_index('day')[['rolling_pcr_pos']]

rolling_pcr.iplot(kind='bar',
              title=title)

Rolling PCR postives for Thursday Dec 3 was 9230, so it's back to the same value as Nov 29. 

What do the projections look like now?

## Update projections with new data

In [107]:
# create new df that now includes rolling avg for past 3 days

q = "jour >= @confined"
new_df = df.copy().query(q).reset_index()
add_rows = spf_updates[['jour','rolling_pcr_pos']].rename(columns={'rolling_pcr_pos':'pos'})
new_df = pd.concat([new_df, add_rows.iloc[1:]]).drop_duplicates().set_index('jour')


# make new projections
metric = 'pos'
target_val = 5000

new_estimated_pos_dates, new_pos_traces = project_target(metric, target_val, new_df)
display(new_estimated_pos_dates)


=== Projected days to reach 5000 pos ===

To meet this, we need to change daily pos by -4305 between now (2020-12-04) and 2020-12-15.
This will require a minimum -0.042 average daily change over next 11 days.



/Users/sarahhosking/anaconda/envs/py36_map/lib/python3.6/site-packages/ipykernel_launcher.py:61: RuntimeWarning:

divide by zero encountered in double_scalars



,Estimate based on previous,Avg. daily change rate,Est. days to reach target,Est. target date
0,14 days,-0.026,18.0,2020-12-22
1,7 days,0.000,-inf,9999-99-99
2,5 days,0.009,-51.0,2020-10-14
3,3 days,0.006,-77.0,2020-09-18


Note the values in the last two columns: **there is no longer a clear path to meeting this target by Dec 15, based on recent trends**.

In [117]:
plot_col = "Positive cases, 7-day avg"
new_df = new_df.rename(columns={'pos': plot_col})
target_trace = pos_target
proj_traces = new_pos_traces

new_pos_fig = plot_progress(plot_col, new_df.reset_index(), target_trace, proj_traces)
new_pos_fig.update_layout(title="<b>{}</b>: <br>Positive cases <b>rising</b> again. ".format(plot_col) +
                          "Impossible to project Dec 15 target using recent trends.")
new_pos_fig.show()

fname = "{}_progress".format(metric)
to_chart_studio(new_pos_fig, fname)

Since 7-day rolling case numbers have begun to go back up, it means that the 7-day change rate is 0, making it impossible to use simple linear calculations to project any date. 

Whereas the 5 and 3-day change rates are positive, which sends the projections _backwards_, i.e. it's now estimating the when we were at 5000 cases _in the past_. 

Recall, **this plot shows only positive PCR tests**. What happens if we add antigen test results to the pcr tests?

## Antigen tests: how many more positives?

In [109]:
title = "Total of 7d-rolling daily PCR postives +  daily antigen positives" 
both_tests = spf_updates.set_index('day')[['rolling_pcr_pos', 'pos_antigen']]

both_tests.iplot(kind='bar',
              barmode='stack',
              title=title)

Now you can see an even clearer upward trend, but more importantly, higher totals. 

You can see that on Dec 3, there were 2039 positive antigen tests. In reality, we are moving further from the 5000 target. 

In [110]:
# how many new daily cases must we shed to get down to 5000?

both_tests.sum(axis=1).subtract(5000).astype('int')

day
Monday       5409
Tuesday      5534
Wednesday    6133
Thursday     6395
Friday       6524
dtype: int64

## Recap

So in reality, we're moving further from our target daily cases, but this is not being commuicated to the public. 

* SPF gives nightly updates, but they do NOT tell us how we're progressing towards the Dec 15 targets. 

* New case numbers are hard to interpret because for PCR tests SPF reports only the **raw** (unsmoothed) numbers for past 24 hours, which vary a lot within a week due to weekend slowdowns in lab results. 

* They also announce the cumulative number of cases since the epidemic began, which is completely useless for understanding _current_ trends and therefore, severity.

* One positive: the SPF includes antigen test results in these nightly totals

### How much can media clarify?

* French media _en direct_ updates invariably echo the numbers from SPF nightly report. Ditto for updates from English-language media like New York Times and The Guardian.

While media increasingly have data journalists like le Monde's _les Décodeurs_ who produce daily graphs that indeed show the trend (the rolling 7-day average), they still face the same issues I've described above:

* These teams rely on the public downloadable data (.csv files) on data.gouv.fr to update their plots quickly and regularly. But the 3-day lag means when SPF announces the numbers for past 24-hours, it will still take 3 more days before these plots include test results for the same time period.


* the downloadable data doesn't include antigen test numbers, while SPF updates do.


* Even Le Monde's les Decodeurs have not yet shown visualizations to specifically answer whether we are on-track to meet the Dec 15 targets. 

As a result, while French data journalists can show visualizations that make the trends easier to discern, the numbers will be almost impossible to match up with the numbers most people are accustomed to reading or hearing nightly. 

Nor do have I found any articles that track progress on Dec 15 targets, or even the recent stagnation of case numbers, in general. 

Indeed, even if the media was doing this, what percentage of the population still reads newspapers? 

<blockquote>Most critically, considering this is, after all, a public health crisis causing enormous societal & economic upheaval, <b>the responsibility for keeping the public informed lies with the government and SPF.</b></blockquote>


### So why isn't there more official info on the Dec 15 targets?

What was the point of stating targets for the loosening of restrictions on national television if le grand public cannot see whether we're on still track to meet them? 

It's especially important now (Dec 4)), when there might still be time to correct course. 

Presumably, the conseil scientifique is indeed tracking this and keeping the government informed. However, why isn't this public? 

~~Even the media isn't talking about this - the focus is on vaccines, and how to manage holiday meals with only 6 people.~~

Update: le Parisien just published an [article on this very topic...behind its paywall](https://www.leparisien.fr/societe/covid-19-les-objectifs-pour-pouvoir-deconfiner-le-15-decembre-seront-ils-atteints-05-12-2020-8412560.php)


All this gives me the impression that the targets were ultimately meaningless, and that the Dec 15 loosening is a _fait accompli_.

### If targets are still relevant, the public needs to be better informed

If I am wrong, and the targets actually do mean something, the French public is in for a very rude surprise. Daily test numbers will have to drop like a rock between now & Dec 15 to meet the 5000 target. 

Yet with the holidays approaching, and more shops open, life feels more "normal", so people are less likely to be vigilant compared to the first weeks of the second confinement.

All the more reason to create a dashboard with a clear visual of where we are today, where we need to get to, and an estimate of how long it will take us to get there. 

**If I can do this from my living room using data.gouv.fr CSV files, open-source python librairies, and basic math, why can't the experts at French government and Santé Publique France?**













# Target 2:  ICU occupancy 2500 - 3000

This still looks good: if we maintain the current ratess, we should get below 3000 in ICU by Dec 8. 

This hopefully leaves enough wiggle room if the decrease slows downs, which is likely: since daily new cases have held steady ~9000 cases for nearly a week, a proportion of those people infected will become seriously ill and end up in the ICU. 

(How big a proportion largely depends on who, or rather how old, are those people getting infected right now)

This in turn will slow and perhaps stop the decrease in ICU occupancy, if not see it increase again.

In [111]:
metric = 'rea'
target_val = 3000

estimated_rea_dates, rea_traces = project_target(metric, target_val, rea_fr)
estimated_rea_dates

=== Projected days to reach 3000 rea ===

To meet this, we need to change daily rea by -259 between now (2020-12-04) and 2020-12-15.
This will require a minimum -0.007 average daily change over next 11 days.



,Estimate based on previous,Avg. daily change rate,Est. days to reach target,Est. target date
0,14 days,-0.019,4,2020-12-08
1,7 days,-0.018,4,2020-12-08
2,5 days,-0.025,3,2020-12-07
3,3 days,-0.019,4,2020-12-08


In [112]:
plot_col = 'Covid patients in ICU'
target_trace = rea_target
proj_traces = rea_traces

rea_fig = plot_progress(plot_col, plot_df, target_trace, proj_traces)
rea_fig.update_layout(title="<b>{}</b>: <br>Currently on track for Dec 15 target".format(plot_col))
rea_fig.show()

fname = "{}_progress".format(metric)
#to_chart_studio(rea_fig, fname)